In [160]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import os
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_recommenders as tfrs
import findspark
findspark.init()

In [161]:
os.environ["SPARK_HOME"]=r"D:\spark\spark"
os.environ["HADOOP_HOME"]=r"D:\spark\spark\hadoop\bin"
os.environ["JAVA_HOME"]=r"D:\spark\jdk-21"

In [162]:
jarsql = r"C:\leads\venv\Lib\site-packages\pyspark\mssql-jdbc-9.4.0.jre11.jar"

In [163]:

from pyspark.sql import SparkSession
from os.path import abspath
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, regexp_replace, when, from_unixtime, lit, concat,to_date,unix_timestamp
import pyspark.sql.functions as F

jarsql = r"C:\leads\venv\Lib\site-packages\pyspark\mssql-jdbc-9.4.0.jre11.jar"
warehouse_location = abspath('spark-warehouse')

conf = SparkConf()
conf.set("spark.master","local[*]")
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set('spark.driver.extraClassPath', jarsql)
conf.set('spark.executor.extraClassPath', jarsql)
spark = SparkSession.builder\
    .config(conf=conf)\
    .config("spark.sql.warehouse.dir", warehouse_location)\
    .appName("Testing PySpark Example")\
    .getOrCreate()


sc = spark.sparkContext

In [164]:
file_path = r"D:\analise_produtos\excelfiles\super_loja.csv"
df = spark.read.option("sep", ";").option("header", "true").option("inferSchema", "true").csv(file_path)

In [165]:
lista_columns_lower = [i for i in df.columns]
for lista in lista_columns_lower:
    
    df = df.withColumnRenamed(
        lista, lista.replace(" ","_").replace(
            "/","_").replace("�o","ao").replace("a�","ai").lower().strip())
df.show()
    

+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|id_da_fila|  id_do_pedido|data_do_pedido|data_de_envio|modo_de_envio|id_do_cliente|  nome_do_cliente|            segmento|          cidade|          estado|pais_regiao| regiao|  id_do_produto|           categoria|sub-categoria|     nome_do_produto|vendas|quantidade|desconto| lucro|
+----------+--------------+--------------+-------------+-------------+-------------+-----------------+--------------------+----------------+----------------+-----------+-------+---------------+--------------------+-------------+--------------------+------+----------+--------+------+
|         1|MX-2021-143658|    02/10/2021|   06/10/2021|Classe padr�o|     SR-20575|      Sofia Rocha|              Varejo|Cidade do M�xico|Distrito

In [166]:
lista_floats = ["vendas","desconto","lucro"]
for col_name in lista_floats:
    df = df.withColumn(col_name, regexp_replace(col(col_name), ",", ".").cast("float"))

In [167]:
date_columns = ["data_do_pedido", "data_de_envio"]
for col_name in date_columns:
    df = df.withColumn(col_name, when(col(col_name) != "NULL", col(col_name)))

In [168]:
date_format = "dd/MM/yyyy"
for col_name in date_columns:
    df = df.withColumn(col_name, to_date(col(col_name), date_format))

In [169]:
tab_vendas = df.select("data_do_pedido","vendas","quantidade","desconto","lucro",
          "data_de_envio","segmento","regiao","categoria","sub-categoria","id_do_produto","nome_do_produto","id_do_pedido","id_do_cliente","estado","cidade")

In [170]:
tab_vendas = tab_vendas.withColumn("quantidade", df["quantidade"].cast("int"))
tab_vendas.show()

dados = tab_vendas.toPandas()

+--------------+------+----------+--------+-------+-------------+--------------------+-------+--------------------+-------------+---------------+--------------------+--------------+-------------+----------------+----------------+
|data_do_pedido|vendas|quantidade|desconto|  lucro|data_de_envio|            segmento| regiao|           categoria|sub-categoria|  id_do_produto|     nome_do_produto|  id_do_pedido|id_do_cliente|          estado|          cidade|
+--------------+------+----------+--------+-------+-------------+--------------------+-------+--------------------+-------------+---------------+--------------------+--------------+-------------+----------------+----------------+
|    2021-10-02| 130.8|         3|     0.0|   45.6|   2021-10-06|              Varejo|  Norte|Material de escri...|    Etiquetas|MAT-ET-10001014|HON Etiquetas par...|MX-2021-143658|     SR-20575|Distrito Federal|Cidade do M�xico|
|    2019-10-16|2521.6|         8|     0.0|  907.2|   2019-10-21|              V

In [171]:
dados

,data_do_pedido,vendas,quantidade,desconto,lucro,data_de_envio,segmento,regiao,categoria,sub-categoria,id_do_produto,nome_do_produto,id_do_pedido,id_do_cliente,estado,cidade
0,2021-10-02,130.800003,3,0.0,45.599998,2021-10-06,Varejo,Norte,Material de escrit�rio,Etiquetas,MAT-ET-10001014,"HON Etiquetas para organiza��o de pastas, Ajus...",MX-2021-143658,SR-20575,Distrito Federal,Cidade do M�xico
1,2019-10-16,2521.600098,8,0.0,907.200012,2019-10-21,Varejo,Sul,M�veis,M�veis,M�V-M�-10004592,"Tenex Rel�gio de parede, Dur�vel",MX-2019-155047,LC-16570,Risaralda,Dos Quebradas
2,2019-10-16,1932.800049,2,0.0,540.799988,2019-10-21,Varejo,Sul,M�veis,Estantes,M�V-ES-10000849,"Ikea Gaveteiro, Branco",MX-2019-155047,LC-16570,Risaralda,Dos Quebradas
3,2019-10-16,354.399994,4,0.0,49.599998,2019-10-21,Varejo,Sul,Material de escrit�rio,Fich�rios,MAT-FI-10003284,"Cardinal Pasta argolada, Transparente",MX-2019-155047,LC-16570,Risaralda,Dos Quebradas
4,2019-10-16,716.000000,2,0.0,114.400002,2019-10-21,Varejo,Sul,Material de escrit�rio,Arte,MAT-AR-10002762,"Sanford Tela, Tamanhos variados",MX-2019-155047,LC-16570,Risaralda,Dos Quebradas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,2019-04-05,74.160004,3,0.6,-81.839996,2019-04-08,Varejo,Sul,Material de escrit�rio,Etiquetas,MAT-ET-10003909,"HON Etiquetas de envio, Ajust�vel",US-2019-145282,LM-17230,Minas Gerais,Ponte Nova
10258,2019-04-05,62.799999,1,0.6,-64.400002,2019-04-08,Varejo,Sul,Material de escrit�rio,Envelopes,MAT-EN-10001954,"Jiffy Fita dupla face, Vermelho",US-2019-145282,LM-17230,Minas Gerais,Ponte Nova
10259,2019-02-11,492.959991,3,0.6,-407.040009,2019-02-15,Pequenas e Medias Empresas,Sul,M�veis,Cadeiras,M�V-CA-10004481,"Harbour Creations Cadeira de praia, Vermelho",US-2019-106523,RA-19795,Alagoas,S�o Miguel dos Campos
10260,2020-11-05,663.520020,2,0.6,-199.279999,2020-11-09,Grande Empresa,Sul,M�veis,Estantes,M�V-ES-10003288,"Ikea Par, Preto",US-2020-168032,BC-11380,Rio de Janeiro,Vassouras


In [172]:
dados = dados.sample(frac=1, random_state=42)
dados["vendas"] = pd.to_numeric(dados["vendas"], errors="coerce").fillna(0)

user_ids = dados["id_do_cliente"].unique()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
dados["id_do_cliente"] = dados["id_do_cliente"].map(user2user_encoded)

produto_ids = dados["id_do_produto"].unique()
produto2produto_encoded = {x: i for i, x in enumerate(produto_ids)}
dados["id_do_produto"] = dados["id_do_produto"].map(produto2produto_encoded)

x = dados[["id_do_cliente", "id_do_produto"]].values
y = dados["vendas"].values
min_sales = dados["vendas"].min()
max_sales = dados["vendas"].max()
y = (y - min_sales) / (max_sales - min_sales)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, random_state=42)

In [ ]:
def train_model(x_train, y_train, num_users, num_produtos, embedding_size):
    class RecommenderNet(keras.Model):
        def __init__(self, num_users, num_produtos, embedding_size, **kwargs):
            super().__init__(**kwargs)
            self.embedding_size = embedding_size
            self.user_embedding = keras.layers.Embedding(
                num_users,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.user_bias = keras.layers.Embedding(num_users, 1)
            self.produto_embedding = keras.layers.Embedding(
                num_produtos,
                embedding_size,
                embeddings_initializer="he_normal",
                embeddings_regularizer=keras.regularizers.l2(1e-6),
            )
            self.produto_bias = keras.layers.Embedding(num_produtos, 1)

        def call(self, inputs):
            user_vector = self.user_embedding(inputs[:, 0])
            user_bias = self.user_bias(inputs[:, 0])
            produto_vector = self.produto_embedding(inputs[:, 1])
            produto_bias = self.produto_bias(inputs[:, 1])
            dot_user_produto = tf.tensordot(user_vector, produto_vector, 2)
            x = dot_user_produto + user_bias + produto_bias
            return x

    model = RecommenderNet(num_users, num_produtos, embedding_size)
    model.compile(
        loss=keras.losses.MeanSquaredError(),
        optimizer=keras.optimizers.Adam(learning_rate=0.01),
    )

    history = model.fit(
        x_train,
        y_train,
        batch_size=64,
        epochs=20,
        verbose=1,
    )

    return model

def get_recommendations_for_user(model, user_id, num_recommendations, produto2produto_encoded, dados):
    user_recommendations = []
   
    user_id = int(user_id)

    user_history = dados[dados['id_do_cliente'] == user_id]

    available_produto_ids = set(produto2produto_encoded.values()) - set(user_history['id_do_produto'].values)

   
    """Gera recomendacoes para usuarios"""
    user_movie_array = np.array([[user_id] * len(available_produto_ids), list(available_produto_ids)], dtype=np.int32)
    ratings = model.predict(user_movie_array).flatten()
    top_ratings_indices = ratings.argsort()[-num_recommendations:][::-1]

    user_recommendations = top_ratings_indices.tolist()

    return user_recommendations

num_users = len(user2user_encoded)
num_produtos = len(produto2produto_encoded)
EMBEDDING_SIZE = 64



model = train_model(x_train, y_train, num_users, num_produtos, EMBEDDING_SIZE)

all_recommendations = []

for user_id in range(num_users):
    user_recommendations = get_recommendations_for_user(model, user_id, 10, produto2produto_encoded, dados)
    all_recommendations.extend(user_recommendations)

for user_id in range(num_users):
    user_recommendations = [product_id for product_id in all_recommendations if produto2produto_encoded.get(product_id) is not None and produto2produto_encoded[product_id] not in dados[dados['id_do_cliente'] == user_id]['id_do_produto'].values][:10]
    print("Recomendações para o usuário:", user_id)
    print("=" * 40)
    for product_id in user_recommendations:
        print(f"Código do Produto: {product_id}, Nome do Produto: {produto2produto_names.get(product_id, 'Produto Desconhecido')}")
    print()



In [ ]:
def get_recommendations_for_all_users(model, num_users, num_recommendations, produto2produto_encoded, encoded2produto, user2user_encoded):
    all_recommendations = []

    for user_id in range(num_users):
        user_movie_array = np.array([[user_id] * len(produto2produto_encoded), list(produto2produto_encoded.values())])
        ratings = model.predict(user_movie_array).flatten()
        top_ratings_indices = ratings.argsort()[-num_recommendations:][::-1]
        recommended_products = [{"id_do_produto": produto_id, "nome_produto": encoded2produto.get(produto_id, "Nome Desconhecido"),
        "id_do_cliente": user_id, "probabilidade": max(0, min(1, ratings[produto_id]))} for produto_id in top_ratings_indices]
        all_recommendations.extend(recommended_products)

    return all_recommendations


num_users = len(user2user_encoded)
num_produtos = len(produto2produto_encoded)
EMBEDDING_SIZE = 64



"""Obtem Recomendacoes"""
encoded2produto = {i: x for x, i in produto2produto_encoded.items()}
all_recommendations = get_recommendations_for_all_users(model, num_users, 10, produto2produto_encoded, encoded2produto, user2user_encoded)


"""Exibe Recomendacoes"""
for user_id in range(num_users):
    user_recommendations = [rec for rec in all_recommendations if rec["id_do_cliente"] == user_id]
    print("Recomendações para o usuário:", user_id)
    print("=" * 40)
    for product in user_recommendations:
        print(f"Código do Produto: {product['id_do_produto']}, Nome do Produto: {product['nome_produto']}, Probabilidade: {product['probabilidade']:.4f}")
    print()


1/1 [==============================] - 0s 17ms/step
Recomendações para o usuário: 0
Código do Produto: 1, Nome do Produto: MAT-EN-10003894, Probabilidade: 0.5842
Código do Produto: 0, Nome do Produto: MAT-FI-10002664, Probabilidade: 0.4314

Recomendações para o usuário: 1
Código do Produto: 1, Nome do Produto: MAT-EN-10003894, Probabilidade: 0.8635
Código do Produto: 0, Nome do Produto: MAT-FI-10002664, Probabilidade: 0.6801

Recomendações para o usuário: 2
Código do Produto: 0, Nome do Produto: MAT-FI-10002664, Probabilidade: 0.6383
Código do Produto: 1, Nome do Produto: MAT-EN-10003894, Probabilidade: 0.6369

Recomendações para o usuário: 3
Código do Produto: 1, Nome do Produto: MAT-EN-10003894, Probabilidade: 0.6084
Código do Produto: 0, Nome do Produto: MAT-FI-10002664, Probabilidade: 0.1827

Recomendações para o usuário: 4
Código do Produto: 1, Nome do Produto: MAT-EN-10003894, Probabilidade: 0.7459
Código do Produto: 0, Nome do Produto: MAT-FI-10002664, Probabilidade: 0.3735

Rec